In [1]:
import dataloader
import numpy as np
import submoddiv as smd
import probit
import pandas as pd
import os

c:\anaconda2\envs\py3.6\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from importlib import reload

reload(probit)
reload(smd)
reload(dataloader)

<module 'dataloader' from 'C:\\Columbia\\Courses\\AdvancedML\\advml\\dataloader.py'>

In [3]:
sparseAdWeights,adIds = dataloader.loadAdWeightsAndIds()
adIdsRev,convertToAdInx = dataloader.getAdIdConverterFunction(adIds)

In [4]:
def findNearestNeighbor(ad, wAds):
    diffs = wAds - ad
    norms = np.linalg.norm(diffs, axis=1)
    newAdIndex = np.argmin(norms)
    #   print(newAdIndex)
    return newAdIndex

In [54]:
def simulate(nUser, wAds, model, userInx, n=6):       
    submod = smd.SubModDivGlobal(nUser, wAds,model)
    
    wAds = wAds.toarray()
    
    subsets = np.empty((0, n))
    prevAds = np.empty(0)
    prevAdData = np.empty((0, wAds.shape[1]))
    print(prevAdData)

    total = np.zeros(wAds.shape[1])
    meanAd = np.zeros(wAds.shape[1])

    for i in range(0, 100):
        subset = submod.getSubSet(userInx, n)
        subsets = np.append(subsets, [subset], axis=0)
        
        newAdIndex = findNearestNeighbor(meanAd, wAds[subset])
        print(subset)
        print(newAdIndex)
        submod.registerClick(subset[newAdIndex])

        prevAds = np.append(prevAds, subset[newAdIndex])
        prevAdData = np.append(prevAdData, [wAds[subset[newAdIndex]]], axis=0)

        total = total + wAds[subset[newAdIndex]]
        meanAd = total/float(prevAdData.shape[0])

#         wAds[newAdIndex, :] = float("inf")
        #     print("NewAdIndex: {}, Total: {}, meanAd: {}".format(newAdIndex, total, meanAd))

    return prevAdData, prevAds, subsets

In [6]:
eventsDf = pd.read_csv('filtered_events.csv')

uniqUser = np.unique(eventsDf.uuid)
nUser = uniqUser.shape[0]

In [7]:
model = probit.createProbitModelGlobal(sparseAdWeights)

In [8]:
weights_filename = 'probit.h5'

if os.path.isfile(weights_filename):
    print("loading existing weights")
    model.load_weights(weights_filename)

loading existing weights


In [44]:
prevAdData.shape

(5, 96)

In [46]:
prevAdData, prevAds, subsets = simulate(nUser, sparseAdWeights, model, 5)
subsets

[]
[ 45609  94016    498  95538 107880  49415]
0
[45609  9523 81141   707  2287   505]
0
[45609 60257    53  1272 12301  5409]
0
[45609 35273   664 51954   784  2765]
0
[ 45609  60257  45211     23 112209    268]
0


array([[4.56090e+04, 9.40160e+04, 4.98000e+02, 9.55380e+04, 1.07880e+05,
        4.94150e+04],
       [4.56090e+04, 9.52300e+03, 8.11410e+04, 7.07000e+02, 2.28700e+03,
        5.05000e+02],
       [4.56090e+04, 6.02570e+04, 5.30000e+01, 1.27200e+03, 1.23010e+04,
        5.40900e+03],
       [4.56090e+04, 3.52730e+04, 6.64000e+02, 5.19540e+04, 7.84000e+02,
        2.76500e+03],
       [4.56090e+04, 6.02570e+04, 4.52110e+04, 2.30000e+01, 1.12209e+05,
        2.68000e+02]])

# Metrics

In [52]:
def accuracyAtK(subsets, adClickInx, nAds, model, userInx, k):
    probs = model.predict([
            np.array([userInx]*nAds),
            np.arange(nAds)
        ],batch_size=50000).ravel()
    
    accuracy = 0
    for i in range(subsets.shape[0]):
        topAds = sorted(subsets[i], key=lambda x: probs[x], reverse=True)[:k]
        if adClickInx[i] in topAds:
            accuracy = accuracy + 1
    
#     topAds = sorted(range(0, len(probs)), key=lambda x: prob[x], reverse=True)[:k]
    
#     common = set(adClickInx).intersection(set(topAds))
#     return len(common)/float(len(adClickInx))

    return accuracy/float(subsets.shape[0])

In [48]:
def meanClickProbability(subSets, model):
    probs = self.model.predict([
            np.array([userInx]*wAds.shape[0]),
            np.arange(wAds.shape[0])
        ],batch_size=50000).ravel()
    
    findSubSetProbs = np.vectorize(lambda subset: np.sum(probs[subset]))
    subSetProbs = findSubSetProbs(subSets)
    totalProb = np.sum(subSetProbs)
    return totalProb/float(len(subSetProbs))

In [5]:
eventsDf.columns

Index(['display_id', 'uuid', 'timestamp', 'ad_id', 'clicked', 'ad_inx',
       'user_inx'],
      dtype='object')

In [10]:
def findAdTrendingProbs(x):
#     print(len(x.loc[x['clicked'] == 1]))
#     print(float(len(x)))
    x['clickProb'] = len(x.loc[x['clicked'] == 1])/float(len(x))
#     print(x)
    return x

In [11]:
groups = eventsDf.groupby(by='ad_inx')

In [12]:
copy = groups.apply(lambda group: findAdTrendingProbs(group))

,display_id,uuid,timestamp,ad_id,clicked,ad_inx,user_inx,clickProb
0,10,b09a0e92aa4d17,693,241662,0,66748,0,0.136364
1,10,b09a0e92aa4d17,693,332908,1,85384,0,0.259259
2,30,53ca43b326638b,2550,139944,1,42405,1,0.286573
3,30,53ca43b326638b,2550,166549,0,49228,1,0.192308
4,30,53ca43b326638b,2550,181548,0,52892,1,0.483051
5,30,53ca43b326638b,2550,250079,0,68169,1,0.008658
6,30,53ca43b326638b,2550,269017,0,71725,1,0.091703
7,79,9d376148017289,5793,79784,0,23656,2,0.000000
8,79,9d376148017289,5793,88711,0,26510,2,0.018750
9,79,9d376148017289,5793,92113,1,27594,2,0.311741


In [13]:
copy = copy[['ad_inx', 'clickProb']].drop_duplicates()
copy

,ad_inx,clickProb
0,66748,0.136364
1,85384,0.259259
2,42405,0.286573
3,49228,0.192308
4,52892,0.483051
5,68169,0.008658
6,71725,0.091703
7,23656,0.000000
8,26510,0.018750
9,27594,0.311741


In [14]:
ad_probs_array = np.zeros(len(copy))
# copy.apply(lambda row: ad_probs_array[row.ad_inx] = row.clickProb)
zipped = zip(copy.ad_inx, copy.clickProb)
for ad_inx, prob in zipped:
    ad_probs_array[ad_inx] = prob
ad_probs_array

array([0.        , 0.        , 0.26804124, ..., 1.        , 0.        ,
       0.        ])

In [15]:
np.unique(ad_probs_array)

array([0.        , 0.00327869, 0.00350877, ..., 0.875     , 0.88888889,
       1.        ])

In [17]:
import _pickle as pickle
pickle.dump(ad_probs_array, open('ad_probs.pickle', 'wb'))

In [59]:
def simTrendingAds(nUser, ad_probs, wAds, model, userInx, n=6):
    submod = smd.SubModDivGlobal(nUser, wAds,model)
    
    wAds = wAds.toarray()
    
    subsets = np.empty((0, n)) 
    prevAds = np.empty(0)
    prevAdData = np.empty((0, wAds.shape[1]))
    print(prevAdData)

#     total = np.zeros(wAds.shape[1])
#     meanAd = np.zeros(wAds.shape[1])

    for i in range(0, 100):
        subset = submod.getSubSet(userInx, n)
        subsets = np.append(subsets, [subset], axis=0)
        
        subsetProbs = ad_probs[subset]
        subsetProbs = subsetProbs/float(sum(subsetProbs))
        
        newAdIndex = selectAdWithProb(subsetProbs)
#         newAdIndex = findNearestNeighbor(meanAd, wAds[subset])
        print(subset)
        print(newAdIndex)
        submod.registerClick(subset[newAdIndex])

        prevAds = np.append(prevAds, subset[newAdIndex])
        prevAdData = np.append(prevAdData, [wAds[subset[newAdIndex]]], axis=0)

#         total = total + wAds[subset[newAdIndex]]
#         meanAd = total/float(prevAdData.shape[0])

#         wAds[newAdIndex, :] = float("inf")
        #     print("NewAdIndex: {}, Total: {}, meanAd: {}".format(newAdIndex, total, meanAd))

    return prevAdData, prevAds, subsets

In [60]:
def selectAdWithProb(ad_probs):
    return np.random.choice(range(0, len(ad_probs)), p=ad_probs)

In [62]:
prevAdData, prevAds, subsets = simTrendingAds(nUser, ad_probs_array, sparseAdWeights, model, 5)
subsets

[]
[ 45609  94016    498  95538 107880  49415]
1
[94016 55696  9523 81141   707  2287]
0
[94016 30130   582 76083  1272  5409]
0
[94016   284 42929 62428 89771  2765]
0
[94016 45211     9  4379    31   790]
0


In [53]:
accuracy = accuracyAtK(subsets, prevAds, sparseAdWeights.shape[0], model, 5, 2)
# accuracyAtK(subsets, adClickInx, wAds, model, userInx, k)
accuracy

ValueError: Error when checking model : the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 2 arrays: [array([[5],
       [5],
       [5],
       ...,
       [5],
       [5],
       [5]]), array([[     0],
       [     1],
       [     2],
       ...,
       [128901],
       [128902],
       [128903]]...